### NYT Digits Puzzle - Algorithm

**(May 23, 2023)**

**Problem Statement:** Can you come up with some code that finds the solution to the NYT digits puzzle given six numbers and a target number? 

In [1]:
from itertools import combinations
from math import comb

In [2]:
# generates all manipulation combinations of a dictionary of elements
def generator_combos(start_dict: dict):
    
    # defining dict_len to determine pathing
    dict_len = len(start_dict)

    # defining output dictionary
    output_dict = dict()
    
    count = 0    
    
    # output the dictionary if there is just one element
    if dict_len==1:
        output_dict = start_dict
    
    # output all ways to add, subtract, multiply, or divide two numbers
    elif dict_len==2:
        # sort to ensure division is >= 1 and subtraction is >= 0
        sorted_vals = dict(sorted(start_dict.items(), key=lambda item: item[1], reverse=True))
        
        # list of names and numbers
        name_list, num_list = list(sorted_vals.keys()), list(sorted_vals.values())    

        # getting elements and there names
        elem1, elem2 = num_list 
        name1, name2 = name_list   

        # set of all possible operations with keys for operation name
        # Note: digits game doesn't allow fractions or negative numbers
        # so we set the result to a large 
        output_dict = {'('+name1+'+'+name2+')':elem1 + elem2, 
                      '('+name1+'*'+name2+')':elem1 * elem2, 
                      '('+name1+'/'+name2+')': int(elem1/elem2) if elem1%elem2==0 else 1/3001, 
                      '('+name1+'-'+name2+')': elem1- elem2 if elem1- elem2>0 else 1/3001 }   

    # for dict_len >=3, recursively build sub solutions
    else: 
        
        # only go up to half the number of elements in dict
        # to prevent double counting (e.g., n choose k = n choose n-k)
        for ix in range(1, dict_len//2+1):

            # number of ways to select 'ix' elements from 'dict_len' total
            num_comb = comb(dict_len, ix)
            
            # limiting number of combinations considered
            # when there is an even split; this prevents double counting
            if ix == dict_len/2:
                lim = num_comb//2
            else:
                lim = num_comb

            # generating combinations of elements
            # '[:lim]' prevents double consideration of combos
            for elem in list(combinations(start_dict.keys(), ix))[:lim]:
                copy_dict  = start_dict.copy() 
                comp_copy_dict = dict() # complement of copy_dict
                for j in range(ix):
                    del copy_dict[elem[j]]
                    comp_copy_dict[elem[j]] = start_dict[elem[j]]
                
                # generating set of possible operations for elements
                for key1, value1 in generator_combos(copy_dict).items():
                    for key2, value2 in generator_combos(comp_copy_dict).items():
                        new_dict = {key1: value1, key2: value2 }
                        result_dict = generator_combos(new_dict)
                        output_dict.update(result_dict) 
                        count += len(result_dict)
                        

    return output_dict

In [3]:
# generator_combos({'1':1, '2': 2, '3': 3, '4': 4, '5':5 })

In [4]:
# for ell in range(1,7):
#     dict_ = {str(num):num for num in range(1,ell+1)}
#     all_combos = generator_combos(dict_)
#     print(f'Number of combinations with {ell} numbers:', len(all_combos))

In [5]:
## function scaffold
def nyt_digits_solver(num_list: list, target: int, soln_num=None):
    
    # generating full list of combinations of all sizes for the input numbers
    soln_set_dict = dict()
    for i in range(2, len(num_list) + 1):  
        for combo in list(combinations(num_list, i)):
            dict_conv = {str(num):num for num in list(combo)}
            temp_soln_dict = generator_combos(dict_conv)
            soln_set_dict.update(temp_soln_dict)
    
    # enumerating all solutions (i.e., combinations that result in target)
    solns = [key for key in soln_set_dict if soln_set_dict[key]==target]
    
    # find the `soln_num` shortest solutions (by char count)
    if soln_num:
        solns_copy = list()
        for k in sorted(solns, key=len)[:soln_num]:
            solns_copy.append(k)
        solns = solns_copy

    return solns

### Examples

In [6]:
# len(nyt_digits_solver(num_list = [3, 5, 9, 13, 20, 23], target=441))

In [8]:
nyt_digits_solver(num_list = [5, 11, 19, 20, 23, 25], 
                  target=413, 
                  soln_num=10)

['(((19*(25-23))*11)-5)',
 '(((11*(25-23))*19)-5)',
 '(((19*11)*(25-23))-5)',
 '(((23*(11+5))+25)+20)',
 '(((23*(11+5))+20)+25)',
 '((23*(11+5))+(25+20))',
 '(((20+(25-23))*19)-5)',
 '((((25+20)-23)*19)-5)',
 '(((25-(23-20))*19)-5)',
 '(((25+23)*(20-11))-19)']

### NYT Digits - Chat GPT

In [17]:
def find_operations(num_list, target):
    def evaluate_expression(expression):
        try:
            return eval(expression)
        except (ZeroDivisionError, TypeError):
            return None

    def backtrack(idx, current_expression):
        if idx == len(num_list):
            result = evaluate_expression(current_expression)
            if result == target:
                return current_expression
            return None

        number = str(num_list[idx])

        # Try all four possible operations with the current number
        expressions = [
            current_expression + "+" + number,
            current_expression + "-" + number,
            current_expression + "*" + number,
            current_expression + "/" + number,
        ]

        for expression in expressions:
            result = backtrack(idx + 1, expression)
            if result:
                return result

        return None

    # Start the backtracking algorithm with an empty expression and index 0
    operations = backtrack(0, "")

    return operations

# Example usage:
num_list = [5, 11, 19, 20, 23, 25]
target=413
result = find_operations(num_list, target)
print(result)


SyntaxError: invalid syntax (<string>, line 1)